In [47]:
import pandas as pd
import os
import zipfile

In [48]:
from config.config_loader import Config

config = Config()

In [49]:
# Parameters
path_alias_elec = config.get_resolved_path("data_path.elec.customers.alias_elec")
path_raw_data_elec = config.get_resolved_path("data_path.elec.invoice_comp.data_raw")
path_processed_files = config.get_path("data_path.elec.invoice_comp.processed_files")
fras_folder = config.get_path("data_path.elec.invoice_comp.invoices.folder")
path_fras_elec_xml_clean = config.get_resolved_path("data_path.elec.invoice_comp.xml_folder_clean")

In [50]:
import yaml

config = Config()

invoice_sftp_config_path = config.get_resolved_path("configs.invoice_comp.sftp_params")
with open(invoice_sftp_config_path, 'r') as file:
    sftp_providers = yaml.safe_load(file)

In [51]:
# # Retrieve sftp invoice files
# from invoice_comp.sftp_helper import download_sftp_files

# for provider, parameter in sftp_providers.items():

#     if parameter['execute'] == True:
#         path_recorded = config.get_path("data_path.elec.invoice_comp.record_sftp_provider", provider=provider)
#         try:
#             download_sftp_files(
#                 parameter['host'],
#                 parameter['port'],
#                 parameter['username'],
#                 parameter['password'],
#                 parameter['remote_dir'],
#                 path_raw_data_elec,
#                 path_recorded
#             )
#         except Exception as e:
#             print(f"Error downloading files from {provider}: {e}")

In [52]:
from invoice_comp.check_utility import extract_cups, extract_month_year, extract_num_fra_elec
import utils.utils_var as utils_var


In [53]:
df_alias_elec = pd.read_excel(path_alias_elec, sheet_name='Hoja1')
df_alias_elec.tail()

,cliente,alias,cups,tarifa,electrointensivo,contrato
59,Grupo Inspired,HQ 1,ES0021000005099278NZ,3.0 TD,False,1
60,Grupo Inspired,HQ 2,ES0021000005099279NS,3.0 TD,False,1
61,PR2,PR2,ES0021000021881819JF,6.1 TD,False,1
62,Kem One,KEM ONE,ES0021000003225511XB,6.2 TD,True,1
63,Generic,TRIAL,TRIAL,6.1 TD,False,1


In [55]:
try:
    df_processed = pd.read_pickle(path_processed_files)
except:
    df_processed = pd.DataFrame(columns=['file_name'])
    
df_processed.head()

,file_name
0,029B689E04E01EEE93AA807352EF4CC7.zip
1,029B689E04E01EEE94F1E7B3F9B60CC7.zip
2,029B689E04E01EEE95D648DC01034CC7.zip
3,029B689E04E01EEE9B979DB3DC32CCC7.zip
4,029B689E04E01EEE9B9D4C2290698CC7.zip


In [ ]:
raw_list = os.listdir(path_raw_data_elec)
fras_list = os.listdir(path_fras_elec_xml_clean)
zip_list = []

for archive in raw_list:

    if '.zip' in archive or '.ZIP' in archive:
        zip_list.append(archive)
    if archive not in df_processed['file_name'].values:
        # print('total')
        if '.xml' in archive or '.XML' in archive:
            num_fra = extract_num_fra_elec(os.path.join(path_raw_data_elec,archive))
            if (not num_fra + '.xml' in fras_list) or (not num_fra + '.XML' in fras_list):
                cups = extract_cups(os.path.join(path_raw_data_elec,archive))
                cliente = utils_var.insert_alias(cups, df_alias_elec)[0]

                if cliente!=None:

                    month, year = extract_month_year(os.path.join(path_raw_data_elec,archive))
                    month = utils_var.map_month_folder(month)
                    print(num_fra)
                    print(cliente)
                    # introducir codigo para ordenar los pdfs
                    with open(os.path.join(path_raw_data_elec, archive), 'r', encoding='utf-8') as input_file:
                        xml_content = input_file.read()  # Read the content of the XML file

                        # Write the XML content to the output path
                        with open(f'{path_fras_elec_xml_clean}/{num_fra}{archive[-4:]}', 'w', encoding='utf-8') as output_file:
                            output_file.write(xml_content)
                            print(f"File saved to: {path_fras_elec_xml_clean}/{num_fra}{archive[-4:]}")

                    if f'{archive[:-4]}.pdf' in raw_list or f'{archive[:-4]}.PDF' in raw_list:

                        if not os.path.exists(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}')):
                            os.makedirs(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}'))

                        with open(os.path.join(f'{path_raw_data_elec}', f'{archive[:-4]}.pdf'), 'rb') as input_file:
                            pdf_content = input_file.read()

                        with open(f'{fras_folder}/{cliente}/{year}/{month}/{num_fra}.pdf', 'wb') as output_file:
                            output_file.write(pdf_content)
                            print(f"File saved to: {fras_folder}/{cliente}/{num_fra}.pdf")
                            
        df_processed = pd.concat([df_processed, pd.DataFrame({'file_name': [archive]})], ignore_index=True)
        df_processed.to_pickle(path_processed_files)

for zip_file in zip_list:

    # if zip_file not in df_processed['file_name'].values:

        print('HI')

        with zipfile.ZipFile(f'{path_raw_data_elec}/{zip_file}', 'r') as zip_ref:
            # List all the files inside the zip
            zip_contents = zip_ref.namelist()
            for i in zip_contents:
                if '.xml' in i or '.XML' in i:

                    with zip_ref.open(i) as file:
                        # Read the contents of the file
                        cups = extract_cups(file)

                    cliente = utils_var.insert_alias(cups, df_alias_elec)[0]

                    if cliente != None:

                        with zip_ref.open(i) as file:
                            # Read the contents of the file
                            num_fra = extract_num_fra_elec(file)
                            print(cups)
                        if (not num_fra + '.xml' in fras_list) or (not num_fra + '.XML' in fras_list):
                            print("Files in zip:", zip_contents)
                            with zip_ref.open(i) as file:
                                month, year = extract_month_year(file)
                                month = utils_var.map_month_folder(month)
                            print({num_fra})

                            with zip_ref.open(i) as file:
                                xml_content = file.read().decode('utf-8')
                            with open(f'{path_fras_elec_xml_clean}/{num_fra}{i[-4:]}', 'w', encoding='utf-8') as output_file:
                                output_file.write(xml_content)
                            
                            for j in zip_contents:
                                if '.pdf' in j:
                                    with zip_ref.open(j) as file:
                                        pdf_content = file.read()  # Read the PDF content in binary mode
                                        # Make sure to use 'wb' mode to write binary content
                                        if not os.path.exists(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}')):
                                            os.makedirs(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}'))
                                        with open(f'{fras_folder}/{cliente}/{year}/{month}/{num_fra}{j[-4:]}', 'wb') as output_file:
                                            output_file.write(pdf_content)